In [ ]:
!pip install qiskit
!pip install qiskit_optimization
!pip install pylatexenc

In [ ]:
from qiskit import *
from qiskit_optimization import *
from qiskit_optimization.algorithms import GroverOptimizer
from qiskit.algorithms import *
import numpy as np
import math

### Let's use Grover's algorithm to maximize the following QUBO function:

$$QUBO = 9x_0x_1 - 6x_0x_2 + 5x_1x_2 - 14x_2 + 11x_0$$

In [ ]:
# First, let's construct the QUBO formulation
qubo = QuadraticProgram()

qubo.binary_var(name='x0')
qubo.binary_var(name='x1')
qubo.binary_var(name='x2')

qubo.maximize(linear={'x0': 11, 'x2': -14}, quadratic={('x0', 'x1'): 9, ('x0', 'x2'): -6, ('x1', 'x2'): 5})

print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 11 x0 - 14 x2 + [ 18 x0*x1 - 12 x0*x2 + 10 x1*x2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1

Binaries
 x0 x1 x2
End



In [ ]:
#Now, let's need to execute the Grover Optimizer function to obtain the minimum optimal value:
backend = Aer.get_backend('qasm_simulator')
# We use 10 iterations and 9 qubits, which is more than enough, to make sure our solution is correct
grover_optimizer = GroverOptimizer(num_value_qubits=9, num_iterations=10, quantum_instance=backend)
results = grover_optimizer.solve(qubo)
print(results.prettyprint())

objective function value: 20.0
variable values: x0=1.0, x1=1.0, x2=0.0
status: SUCCESS


### We can verify that the algorithm is working correctly using the MinimumEigenOptimizer in Qiskit.

In [ ]:
from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
exact_result = exact_solver.solve(qubo)
print(exact_result.prettyprint())

objective function value: 20.0
variable values: x0=1.0, x1=1.0, x2=0.0
status: SUCCESS


# Let's use the FakeVigo simulator to perform the same operation and see consistency

In [ ]:
from qiskit.providers.fake_provider import FakeVigo
from qiskit_aer import AerSimulator

backend = AerSimulator.from_backend(FakeVigo())
grover_optimizer = GroverOptimizer(num_value_qubits=1, num_iterations=20, quantum_instance=backend)
results = grover_optimizer.solve(qubo)
print(results.prettyprint())

objective function value: 11.0
variable values: x0=1.0, x1=0.0, x2=0.0
status: SUCCESS


#### The vigo simulator only provides 5 qubits, less than the QASM used in the previous experiment and it was built to run noisy simulations on quantum circuits, differently of QASM, which runs a noiseless simulation.
#### The result throws an erroneous maximum of 5.
#### This is because of the simulated noise and because the amount of qubits used is not enough to assure a good solution will be found.